In [ ]:
#https://developers.generativeai.google/api/python/google/generativeai
!pip install google-generativeai

In [ ]:
import re
import time
import google.generativeai as palm
import pandas as pd
import pandas as pd
import os
palm.configure(api_key="your api key here")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Chnage working directory

In [ ]:
%cd "/content/drive/MyDrive/ABSA work/codes/bard/"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-995035e8fdf6>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', '"/content/drive/MyDrive/ABSA work/codes/bard/"')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
%pwd

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-d3dc51b95a25>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('pwd', '')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-84>", line 2, in pwd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 295, in pwd
    return os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 20

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-d3dc51b95a25>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('pwd', '')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-84>", line 2, in pwd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 295, in pwd
    return os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 20

In [ ]:
defaults = {
  'model': 'models/text-bison-001',
  'temperature': 0.2,
  'candidate_count': 1,
  'top_k': 40,
  'top_p': 0.95,
  'max_output_tokens': 1024,
  'stop_sequences': [],
  'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":1},{"category":"HARM_CATEGORY_TOXICITY","threshold":1},{"category":"HARM_CATEGORY_VIOLENCE","threshold":2},{"category":"HARM_CATEGORY_SEXUAL","threshold":2},{"category":"HARM_CATEGORY_MEDICAL","threshold":2},{"category":"HARM_CATEGORY_DANGEROUS","threshold":2}],
}

# Load Dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ACSA_test.csv")
df = df.dropna(axis=1, how='all')
print(len(df))
df.head(2)

400


,sentence,aspect_categories,polarity
0,"We went again and sat at the bar this time, I ...","['place', 'food']","['neutral', 'negative']"
1,"The food was good, but it's not worth the wait...","['food', 'service']","['positive', 'negative']"


# Set path for saving the results

In [ ]:
path = "results/MAMS/"
filename="ACSA_test"

In [ ]:

if(os.path.isfile(path+filename+"_bard.csv")):
    temp = pd.read_csv(path+filename+"_bard.csv")
else:
    temp=[]
print(len(temp))


400


In [ ]:
feedback=[]
keywords= [ 'sentiment']
results = {rubric: [] for rubric in keywords}
for i in range(len(temp), len(df)):
  data = df[i:i+1]
  for index, row in data.iterrows():
    # Get the text to assess
    input = row["sentence"]
    aspects = row["aspect_categories"]
    response = predict_sentiment(input, aspects)
    print(type(response))
    feedback.append(str(response))
    print(len(feedback))
    for i in range(len(keywords)):
        keyword = keywords[i]
        pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
        match = pattern.search(response)
        if match:
            results[keyword].append(match.group(1).strip()[2:].lower())
        else:
            results[keyword].append("")
  # data['Response_bard'] = feedback
  data.loc[:, "Response_bard"] = feedback
  print(results['sentiment'])
  for keyword in keywords:
    data.loc[:, keyword+"_bard"] = results[keyword]
  print(response)
  print(row)
  data.to_csv(path+filename+"_bard.csv", index = False, header=False, mode='a')
  # data.to_csv(path+filename+"_bard.csv", index = False)
  results = {rubric: [] for rubric in keywords}
  feedback=[]
  time.sleep(20)

<class 'str'>
1
["['positive', 'positive', 'positive']"]
Sentiment: ['Positive', 'Positive', 'Positive']
sentence        The food was served promptly but the meal wasn...
aspect_terms                     ['food', 'served', 'appetizers']
polarity                      ['neutral', 'positive', 'positive']
Name: 0, dtype: object


<ipython-input-13-c422cf790c96>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Response_bard"] = feedback
<ipython-input-13-c422cf790c96>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, keyword+"_bard"] = results[keyword]


<class 'str'>
1
["['positive', 'positive', 'positive']"]
Sentiment: ['Positive', 'Positive', 'Positive']
sentence        When I got home, there was a message on the ma...
aspect_terms                        ['owner', 'waitress', 'wine']
polarity                       ['neutral', 'negative', 'neutral']
Name: 1, dtype: object


<ipython-input-13-c422cf790c96>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Response_bard"] = feedback
<ipython-input-13-c422cf790c96>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, keyword+"_bard"] = results[keyword]


<class 'str'>
1
["['positive', 'positive']"]
Sentiment: ['Positive', 'Positive']
sentence        The Scene Shun Lee Palace is popular with midt...
aspect_terms                                   ['Scene', 'lunch']
polarity                                  ['positive', 'neutral']
Name: 2, dtype: object


...

<class 'str'>
1
["['negative', 'positive']"]
Sentiment: ['Negative', 'Positive']
sentence        Main courses we had (beef, lobster sukiyaki, b...
aspect_terms                             ['beef', 'Chinese food']
polarity                                  ['neutral', 'positive']
Name: 469, dtype: object


<ipython-input-13-c422cf790c96>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Response_bard"] = feedback
<ipython-input-13-c422cf790c96>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, keyword+"_bard"] = results[keyword]


<class 'str'>
1
["['positive', 'positive', 'positive', 'positive', 'positive', 'positive']"]
Sentiment: ['Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive']
sentence        I paid $12 including tax tip for a beef entree...
aspect_terms    ['tip', 'beef entree with salad', 'noodles', '...
polarity        ['neutral', 'neutral', 'neutral', 'neutral', '...
Name: 470, dtype: object


<ipython-input-13-c422cf790c96>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Response_bard"] = feedback
<ipython-input-13-c422cf790c96>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, keyword+"_bard"] = results[keyword]


KeyboardInterrupt: ignored

In [ ]:
feedback=[]
keywords= [ 'sentiment']
results = {rubric: [] for rubric in keywords}
for i in range(len(temp), len(df)):
  data = df[i:i+1]
  for index, row in data.iterrows():
    # Get the text to assess
    input = row["sentence"]
    aspects = row["aspect_categories"]
## joint task this prompt will find out the aspect and sentiments at a same time
#     prompt = f"""As an aspect-based sentiment analyzer, your task is to extract aspects and corresponding sentiments (Positive, Negative, or Neutral) from people's reviews. Each review consists of a text and may contain multiple aspects and their corresponding sentiments. Your goal is to process the reviews and extract the aspects and sentiments from them.

# Please analyze the provided review below and present the aspects and sentiments in the specified format. Ensure that the aspects you provide show some relation and limit them to the most important aspects in the review. Please do not explain any aspect or sentiment. I do not need any extra text. Just the lists of aspects and the sentiments.

# Review: {input}

# Required output Format:

# aspect: ['aspect1', 'aspect2', 'aspect3', ...]
# Sentiment: ['sentiment1', 'sentiment2', 'sentiment3', ..]"""
    prompt = f"""As an aspect-based sentiment analyzer, your task is to extract sentiments (Positive, Negative, or Neutral) from people's review for a given list of aspects.
    Each review consists of a text and may contain single or multiple aspects and their corresponding sentiments. You will be provided with the review and the list of aspects.
    Your goal is to process the reviews and given list of aspects. Then, predict sentiment of each given aspect.

Please present the result in specified format.
Please do not explain any aspect or sentiment. I do not need any extra text. Just the lists of sentiments.

Review: {input}
Aspects: {aspects}
Required output Format:
Sentiment: ['sentiment1', 'sentiment2', 'sentiment3', ..]"""


    response = palm.generate_text(
      **defaults,
      prompt=prompt
    )
    print(response.result)
    print(type(response.result))
    feedback.append(str(response.result))
    print(len(feedback))
    for i in range(len(keywords)):
        keyword = keywords[i]
        pattern = re.compile(rf"(?i)\b{keyword}\b(.+?)(?=\b{keywords[i+1]}\b|$)" if i < len(keywords) - 1 else rf"(?i)\b{keyword}\b(.+?$)", re.DOTALL)
        match = pattern.search(response.result)
        if match:
            results[keyword].append(match.group(1).strip()[2:].lower())
        else:
            results[keyword].append("")
  # data['Response_bard'] = feedback
  data.loc[:, "Response_bard"] = feedback
  print(results['sentiment'])
  for keyword in keywords:
    data.loc[:, keyword+"_bard"] = results[keyword]
  print(response.result)
  print(row)
  data.to_csv(path+filename+"_bard.csv", index = False, header=False, mode='a')
  results = {rubric: [] for rubric in keywords}
  feedback=[]
  time.sleep(20)

In [ ]:
tem= pd.read_csv(path+filename+"_bard.csv")
tem.head(10)

,sentence,aspect_categories,polarity,Response_bard,sentiment_bard
0,"We went again and sat at the bar this time, I ...","['place', 'food']","['neutral', 'negative']","Sentiment: ['Negative', 'Negative']","['negative', 'negative']"
1,"The food was good, but it's not worth the wait...","['food', 'service']","['positive', 'negative']","Sentiment: ['Positive', 'Negative']","['positive', 'negative']"
2,Waiter took our drink order and then we didn't...,"['staff', 'miscellaneous']","['negative', 'neutral']","Sentiment: ['Negative', 'Negative']","['negative', 'negative']"
3,It does get crowded with plenty of blue shirte...,"['food', 'miscellaneous']","['negative', 'neutral']","Sentiment: ['Neutral', 'Positive']","['neutral', 'positive']"
4,"After hearing all of the specials, you would t...","['food', 'price']","['neutral', 'negative']","Sentiment: ['Positive', 'Negative']","['positive', 'negative']"
5,Our waitress took our order and then NEVER cam...,"['staff', 'miscellaneous']","['negative', 'neutral']","Sentiment: ['Negative', 'Negative']","['negative', 'negative']"
6,"When we sat down, the waiter barely looked in ...","['staff', 'menu']","['negative', 'neutral']","Sentiment: ['Negative', 'Neutral']","['negative', 'neutral']"
7,"they continue to expand their garden space, w/...","['place', 'service']","['positive', 'negative']","Sentiment: ['Negative', 'Negative']","['negative', 'negative']"
8,Extremely RUDE servers who intially took our o...,"['staff', 'food']","['negative', 'neutral']","Sentiment: ['Negative', 'Neutral']","['negative', 'neutral']"
9,1) Service is very slow and unattentive (10 mi...,"['service', 'staff', 'menu', 'food']","['negative', 'negative', 'neutral', 'neutral']","Sentiment: ['Negative', 'Negative', 'Neutral',...","['negative', 'negative', 'neutral', 'neutral']"
